In [1]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset,DataLoader

link='C:/Users/NavneethBhat/Desktop/Code/Implementations/data/train.csv/train.csv'

class DigitDataset(Dataset):
    def __init__(self,link):
        dataset=pd.read_csv(link,dtype=np.float32)
        y=dataset['label'].values
        x=dataset.drop('label',axis=1).values
        self.x=torch.from_numpy(x)
        self.y=torch.from_numpy(y)
        
    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
    
    def __len__(self):
        return self.x.shape[0]
    
data=DigitDataset(link)

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes=10
num_epochs=6
batch_size=1
learning_rate=2e-5
load_model=True
train_loader=DataLoader(dataset=data,batch_size=batch_size,shuffle=True)

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1=nn.Conv2d(1,6,3)
        self.pool=nn.MaxPool2d(2,2)
        self.fc1=nn.Linear(6*13*13,10)
    
    def forward(self,x):
        x=self.pool(self.c1(x))
        x=torch.flatten(x,1)
       # print(x.shape)
        x=self.fc1(x)
       # print(x.shape)
        return x

In [3]:
def save(checkpoint,filename='saved.pth.tar'):
    print("Saving")
    torch.save(state,filename)

In [4]:
def load(checkpoint):
    print("Loading")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [5]:
model=Net()
model.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [7]:
#if load_model:
    #load(torch.load('saved.pth.tar'))

In [8]:
for epoch in range(num_epochs):
    if (epoch%2)==0:
        state={"state_dict":model.state_dict(),"optimizer":optimizer.state_dict()}
        save(state)
    for i,(images,labels) in enumerate(train_loader):
        labels=labels.type(torch.LongTensor)
        images=images.view(1,1,28,28)
        
        images=images.to(device)
        labels=labels.to(device)
        
        optimizer.zero_grad()
        preds=model(images)

        loss=criterion(preds,labels)
        loss.backward()
        optimizer.step()

    print(loss)

Saving
tensor(5.9605e-07, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.1921e-07, device='cuda:0', grad_fn=<NllLossBackward0>)
Saving


KeyboardInterrupt: 

In [20]:
import matplotlib.pyplot as plt
image,label=next(iter(train_loader))
with torch.no_grad():
    image=image.view(1,1,28,28)
    image=image.to(device)
    pred=model(image)
    print(torch.argmax(pred))
    print(label)

tensor(3, device='cuda:0')
tensor([3.])
